In [2]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/{}/{}'
import requests
headers = {
    'Accept': 'application/json',
    'token': 'ItZCKLAcQgyDWSAapgKlybUZGZbqGVNx'
}
import pandas as pd

In [3]:
datasets = requests.get(url.format('datasets', '?limit=1000'), headers = headers)

In [4]:
datasets = pd.DataFrame(datasets.json()['results'])

In [6]:
datasets

,datacoverage,id,maxdate,mindate,name,uid
0,1.00,GHCND,2019-05-08,1763-01-01,Daily Summaries,gov.noaa.ncdc:C00861
1,1.00,GSOM,2019-04-01,1763-01-01,Global Summary of the Month,gov.noaa.ncdc:C00946
2,1.00,GSOY,2018-01-01,1763-01-01,Global Summary of the Year,gov.noaa.ncdc:C00947
3,0.95,NEXRAD2,2019-05-09,1991-06-05,Weather Radar (Level II),gov.noaa.ncdc:C00345
4,0.95,NEXRAD3,2019-05-07,1994-05-20,Weather Radar (Level III),gov.noaa.ncdc:C00708
5,1.00,NORMAL_ANN,2010-01-01,2010-01-01,Normals Annual/Seasonal,gov.noaa.ncdc:C00821
6,1.00,NORMAL_DLY,2010-12-31,2010-01-01,Normals Daily,gov.noaa.ncdc:C00823
7,1.00,NORMAL_HLY,2010-12-31,2010-01-01,Normals Hourly,gov.noaa.ncdc:C00824
8,1.00,NORMAL_MLY,2010-12-01,2010-01-01,Normals Monthly,gov.noaa.ncdc:C00822
9,0.25,PRECIP_15,2014-01-01,1970-05-12,Precipitation 15 Minute,gov.noaa.ncdc:C00505


In [10]:
dataCate = requests.get(url.format('datacategories', '?datasetid=GSOM&limit=1000'), headers = headers)

In [11]:
dataCate = pd.DataFrame(dataCate.json()['results'])

In [12]:
dataCate

,id,name
0,EVAP,Evaporation
1,LAND,Land
2,PRCP,Precipitation
3,SKY,Sky cover & clouds
4,SUN,Sunshine
5,TEMP,Air Temperature
6,WATER,Water
7,WIND,Wind
8,WXTYPE,Weather Type


In [77]:
dataType = requests.get(url.format('datatypes', \
                                   '?datasetid=GSOM&datacategoryid=WIND&limit=1000'), headers = headers)

In [78]:
dataType = pd.DataFrame(dataType.json()['results'])

In [61]:
location_cate = requests.get(url.format('locationcategories', ''), headers = headers)

location_cate = pd.DataFrame(location_cate.json()['results'])

In [62]:
location_cate

,id,name
0,CITY,City
1,CLIM_DIV,Climate Division
2,CLIM_REG,Climate Region
3,CNTRY,Country
4,CNTY,County
5,HYD_ACC,Hydrologic Accounting Unit
6,HYD_CAT,Hydrologic Cataloging Unit
7,HYD_REG,Hydrologic Region
8,HYD_SUB,Hydrologic Subregion
9,ST,State


In [6]:
location = requests.get(url.format('locations', '?datasetid=GSOM&locationcategoryid=ST&limit=1000'), headers = headers)
location = pd.DataFrame(location.json()['results'])

In [7]:
location.loc[location['name'].str.contains('Dakota')]

,datacoverage,id,maxdate,mindate,name
34,1,FIPS:38,2019-04-01,1893-01-01,North Dakota
41,1,FIPS:46,2019-04-01,1893-01-01,South Dakota


The necessary dataTypeID:
for precipitation:PRCP
for sun:TSUN
for TEMP: TAVG
for wind: AWND


Western Corn States:
Lowa IA  FIPS:19
Nebraska NE FIPS:31
Minnesota    MN FIPS:27
Kansas    KS FIPS:20
South Dakota    SD FIPS:46
North Dakota    ND FIPS:38

In [171]:
stations_prcp = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=PRCP&startdate={}-01-01&enddate={}-12-31&limit=1000'\
                                            .format('FIPS:19', 1999, 2008)), headers = headers)                                             

In [13]:
location_id = {
    'illinois' : 'FIPS:17',
    'ohio' : 'FIPS:39',
    'indiana' : 'FIPS:18',
    'wisconsin' : 'FIPS:55',
    'missouri' : 'FIPS:29',
    'michigan' : 'FIPS:26',
    'oklahoma' : 'FIPS:40',
    'texas' : 'FIPS:48',
    'north carolina' : 'FIPS:37',
    'iowa' : 'FIPS:19',
    'nebraska' : 'FIPS:31',
    'minnesota' : 'FIPS:27',
    'kansas' : 'FIPS:20',
    'south dakota' : 'FIPS:46',
    'north dakota' : 'FIPS:38' 
}
def data_generator(sty, edy, state):
    #The prcp data
    stations_prcp = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=PRCP&startdate={}-01-01&enddate={}-12-31&limit=1000'\
                                            .format(location_id[state], sty, edy)), headers = headers)

    stations_prcp = pd.DataFrame(stations_prcp.json()['results'])

    final_res_prcp = pd.DataFrame([])
    for item in list(stations_prcp['id']):
        item = item[6:]
        try:
            mytest = pd.read_csv('/Users/yuying/cropPlus/gsom-latest/{}.csv'.format(item))
            mytest = mytest[['DATE', 'PRCP']]
        except:
            print('{} skip one station {}'.format(state, item))
            continue
        final_res_prcp = pd.concat([mytest, final_res_prcp], sort = False)
    final_res_prcp['DATE'] = pd.to_datetime(final_res_prcp['DATE'], format = '%Y-%m')
    final_res_prcp = final_res_prcp.loc[(final_res_prcp['DATE'] > '{}-12-31'.\
                                         format(sty-1))&(final_res_prcp['DATE'] < '{}-01-01'.format(edy+1))]
    final_res_prcp.dropna(inplace = True)
    final_res_prcp = final_res_prcp.groupby('DATE').mean()
    final_res_prcp.reset_index(drop = True, inplace = True)
    #The temp data
    stations_TAVG = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=TAVG&startdate={}-01-01&enddate={}-12-31&limit=1000'.\
                                            format(location_id[state], sty, edy)), headers = headers)

    stations_TAVG = pd.DataFrame(stations_TAVG.json()['results'])
    final_res_tavg = pd.DataFrame([])
    for item in list(stations_TAVG['id']):
        item = item[6:]
        try:
            mytest = pd.read_csv('/Users/yuying/cropPlus/gsom-latest/{}.csv'.format(item))
            mytest = mytest[['DATE', 'TAVG']]
        except:
            print('{} skip one station {}'.format(state, item))
            continue
        final_res_tavg = pd.concat([mytest, final_res_tavg], sort = False)

    final_res_tavg = final_res_tavg.loc[(final_res_tavg['DATE'] > '{}-12'.\
                                         format(sty-1))&(final_res_tavg['DATE'] < '{}-01'.format(edy+1))]
    final_res_tavg.dropna(inplace = True)
    final_res_tavg = final_res_tavg.groupby('DATE').mean()
    final_res_tavg.reset_index(drop = True, inplace = True)


    #The awnd data
    stations_AWND = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=AWND&startdate={}-01-01&enddate={}-12-31&limit=1000'.\
                                           format(location_id[state], sty, edy)), headers = headers)

    stations_AWND = pd.DataFrame(stations_AWND.json()['results'])
    final_res_awnd = pd.DataFrame([])
    for item in list(stations_AWND['id']):
        item = item[6:]
        try:
            mytest = pd.read_csv('/Users/yuying/cropPlus/gsom-latest/{}.csv'.format(item))
            mytest = mytest[['DATE', 'AWND']]
        except:
            print('{} skip one station {}'.format(state, item))
            continue
        final_res_awnd = pd.concat([mytest, final_res_awnd], sort = False)

    final_res_awnd = final_res_awnd.loc[(final_res_awnd['DATE'] > '{}-12'.\
                                        format(sty-1))&(final_res_awnd['DATE'] < '{}-01'.format(edy+1))]
    final_res_awnd.dropna(inplace = True)
    final_res_awnd = final_res_awnd.groupby('DATE').mean()
    final_res_awnd.reset_index(drop = True, inplace = True)
    final_res_awnd['AWND'] = final_res_awnd['AWND'].fillna(0)


    final_res = pd.concat([final_res_prcp, final_res_tavg, final_res_awnd], axis = 1)

    daterange1 = pd.date_range('{}-01'.format(sty), '{}-01'.format(edy+1), freq='M')
    daterange1 = list(daterange1.strftime('%Y-%m-%d'))

    final_res['DATE'] = daterange1
    final_res['DATE'] = final_res['DATE'].apply(str)

    final_res_diff = pd.DataFrame([])
    for i in range(1, 13, 1):
        if i < 10:
             i = '0' + str(i)
        else:
            i = str(i)
        temp = final_res.loc[final_res['DATE'].str.contains('-{}-'.format(i))]
        temp_slice = temp[['PRCP', 'TAVG', 'AWND']]

        temp_res = temp_slice - temp_slice.iloc[0]

        temp_res['DATE'] = temp['DATE']

        final_res_diff = pd.concat([temp_res, final_res_diff], sort = False)

    new_final = pd.DataFrame(list(range(sty, edy+1)), columns = ['DATE'])

    for yr in list(range(sty, edy+1)):
        for i in range(1, 13, 1):
            if i < 10:
                 i = '0' + str(i)
            else:
                i = str(i)
            my_temp1 = final_res_diff.loc[final_res_diff['DATE'].str.contains('{}-{}-'.format(yr, i))]['PRCP']
            new_final.loc[new_final['DATE'] == yr, 'delta_prcp_{}'.format(i)] = round(float(my_temp1), 2)
            my_temp2 = final_res_diff.loc[final_res_diff['DATE'].str.contains('{}-{}-'.format(yr, i))]['TAVG']
            new_final.loc[new_final['DATE'] == yr, 'delta_tavg_{}'.format(i)] = round(float(my_temp2), 2)
            my_temp3 = final_res_diff.loc[final_res_diff['DATE'].str.contains('{}-{}-'.format(yr, i))]['AWND']
            new_final.loc[new_final['DATE'] == yr, 'delta_awnd_{}'.format(i)] = round(float(my_temp3), 2)


    corn_yield = pd.read_csv('corn_yield_{}_{}_{}.csv'.format(state, sty, edy))
    corn_yield = corn_yield[['Year', 'Value']]

    corn_yield.sort_values(by = ['Year'], inplace = True)

    corn_yield['Value'] = corn_yield['Value'] - corn_yield['Value'].iloc[0]

    new_final = new_final.merge(corn_yield, left_on = 'DATE', right_on = 'Year')

    new_final.drop(axis = 1, columns = ['Year'], inplace = True)
    new_final.rename(columns = {'Value' : 'delta_yield_corn_{}'.format(state)}, inplace = True)

    new_final.to_csv('final_data_combined_yield_{}_{}_{}.csv'.format(state, sty, edy), index = False)

In [8]:
states = list(location_id.keys())

In [9]:
raw = pd.read_csv('corn_yield_raw.csv')

In [15]:
for state in states:
    temp1 = raw.loc[(raw['State'] == state.upper())&(raw['Year']>2008)]
    temp2 = raw.loc[(raw['State'] == state.upper())&(raw['Year']<2009)]
    temp2.to_csv('corn_yield_{}_1999_2008.csv'.format(state), index = False)
    temp1.to_csv('corn_yield_{}_2009_2018.csv'.format(state), index = False)

In [16]:
for state in states:
    data_generator(1999, 2008, state)
    data_generator(2009, 2018, state)

illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILWN0002
illinois skip one station US1ILBU0005
illinois skip one station US1ILCP0064
illinois skip one station US1ILCP0121
illinois skip one station US1ILDG0004
illinois skip one station US1ILDW0008
illinois skip one station US1ILEF0029
illinois skip one station US1ILHM0003
illinois skip one station US1ILJF0010
illinois skip one station US1ILJH0001
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILME0008
illinois skip one station US1ILWL0148
illinois skip one station US1ILWL0149
ohio skip one station US1OHAL0010
ohio skip one station US1OHBL0006
ohio skip one station US1OHFF0014
ohio skip one station US1OHFR0083
ohio skip one station US1OHFR0084
ohio skip one station US1OHHM0037
ohio skip one station US1OHJF0009
ohio skip one station US1OHMK0012
ohio skip one station US1OHOT0009
ohio skip 

minnesota skip one station US1MNHN0213
minnesota skip one station US1MNPS0006
minnesota skip one station US1MNSL0139
minnesota skip one station US1MNSR0032
minnesota skip one station US1MNWG0066
kansas skip one station US1KSCL0002
kansas skip one station US1KSWB0003
kansas skip one station US1KSBR0002
kansas skip one station US1KSBR0004
kansas skip one station US1KSBR0005
kansas skip one station US1KSBU0034
kansas skip one station US1KSCL0002
kansas skip one station US1KSCY0004
kansas skip one station US1KSCY0015
kansas skip one station US1KSDG0063
kansas skip one station US1KSDG0064
kansas skip one station US1KSDP0016
kansas skip one station US1KSEL0095
kansas skip one station US1KSLV0010
kansas skip one station US1KSMN0025
kansas skip one station US1KSMS0005
kansas skip one station US1KSNM0006
kansas skip one station US1KSOS0023
kansas skip one station US1KSPR0018
kansas skip one station US1KSRA0019
kansas skip one station US1KSRA0025
kansas skip one station US1KSRH0015
kansas skip o

In [17]:
all_data_1999 = {}
all_data_2009 = {}
for state in states:
    all_data_1999[state] = pd.read_csv('final_data_combined_yield_{}_1999_2008.csv'.format(state))
    all_data_2009[state] = pd.read_csv('final_data_combined_yield_{}_2009_2018.csv'.format(state))

In [18]:
corr_1999 = {}
corr_2009 = {}
for state in states:
    corr_1999[state] = all_data_1999[state].iloc[:, 1:].corr()
    corr_2009[state] = all_data_2009[state].iloc[:, 1:].corr()


#prcp
lowa_prcp_1999_2008 = lowa_1999_2008.filter(like = 'prcp', axis = 1).copy()
lowa_prcp_1999_2008['delta_yield_corn_lowa'] = lowa_1999_2008.iloc[:, -1]
kansas_prcp_1999_2008 = kansas_1999_2008.filter(like = 'prcp', axis = 1).copy()
kansas_prcp_1999_2008['delta_yield_corn_kansas'] = kansas_1999_2008.iloc[:, -1]
m_prcp_1999_2008 = minnesota_1999_2008.filter(like = 'prcp', axis = 1).copy()
m_prcp_1999_2008['delta_yield_corn_minnesota'] = minnesota_1999_2008.iloc[:, -1]
n_prcp_1999_2008 = nebraska_1999_2008.filter(like = 'prcp', axis = 1).copy()
n_prcp_1999_2008['delta_yield_corn_nebraska'] = nebraska_1999_2008.iloc[:, -1]
nd_prcp_1999_2008 = north_dakota_1999_2008.filter(like = 'prcp', axis = 1).copy()
nd_prcp_1999_2008['delta_yield_corn_north dakota'] = north_dakota_1999_2008.iloc[:, -1]
sd_prcp_1999_2008 = south_dakota_1999_2008.filter(like = 'prcp', axis = 1).copy()
sd_prcp_1999_2008['delta_yield_corn_south dakota'] = south_dakota_1999_2008.iloc[:, -1]
#tavg
lowa_prcp_1999_2008 = lowa_1999_2008.filter(like = 'tavg', axis = 1).copy()
lowa_prcp_1999_2008['delta_yield_corn_lowa'] = lowa_1999_2008.iloc[:, -1]
kansas_prcp_1999_2008 = kansas_1999_2008.filter(like = 'tavg', axis = 1).copy()
kansas_prcp_1999_2008['delta_yield_corn_kansas'] = kansas_1999_2008.iloc[:, -1]
m_prcp_1999_2008 = minnesota_1999_2008.filter(like = 'tavg', axis = 1).copy()
m_prcp_1999_2008['delta_yield_corn_minnesota'] = minnesota_1999_2008.iloc[:, -1]
n_prcp_1999_2008 = nebraska_1999_2008.filter(like = 'tavg', axis = 1).copy()
n_prcp_1999_2008['delta_yield_corn_nebraska'] = nebraska_1999_2008.iloc[:, -1]
nd_prcp_1999_2008 = north_dakota_1999_2008.filter(like = 'tavg', axis = 1).copy()
nd_prcp_1999_2008['delta_yield_corn_north dakota'] = north_dakota_1999_2008.iloc[:, -1]
sd_prcp_1999_2008 = south_dakota_1999_2008.filter(like = 'tavg', axis = 1).copy()
sd_prcp_1999_2008['delta_yield_corn_south dakota'] = south_dakota_1999_2008.iloc[:, -1]

In [28]:
import plotly.graph_objs as go
import plotly
import os

In [30]:
#plot for 1999_2008
def plot_1999(states, area):
    for fc in ['prcp', 'tavg', 'awnd']:
        X_axis = list(corr_1999['iowa'].filter(like = fc, axis = 0).index)
        #'lowa', 'kansas', 'minnesota', 
        data = []
        for state in states:
            trace = go.Scatter(x = X_axis, y = corr_1999[state]['delta_yield_corn_{}'.format(state)].filter(like = fc)\
                            , name = '{}_1999_2008'.format(state))
            data.append(trace)
        layout = dict(title = 'corn correlation show for different states', xaxis = dict(title = 'factors'), \
                      yaxis = dict(title = 'correlations between yield change and factors'))
        fig = dict(data = data, layout = layout)
        if not os.path.exists(os.path.dirname('{}/1999_2008/'.format(area))):
                os.makedirs(os.path.dirname('{}/1999_2008/'.format(area)))
        plotly.offline.plot(fig, filename = '{}/1999_2008/corn correlation show for {}_1999.html'.format(area, fc),\
                            auto_open = False)

In [31]:
#plot for 2009_2018
def plot_2009(states, area):
    for fc in ['prcp', 'tavg', 'awnd']:
        X_axis = list(corr_2009['iowa'].filter(like = fc, axis = 0).index)
        data = []
        for state in states:
            trace = go.Scatter(x = X_axis, y = corr_2009[state]['delta_yield_corn_{}'.format(state)].filter(like = fc)\
                            , name = '{}_2009_2018'.format(state))
            data.append(trace)
        layout = dict(title = 'corn correlation show for different states', xaxis = dict(title = 'factors'), \
                      yaxis = dict(title = 'correlations between yield change and factors'))
        fig = dict(data = data, layout = layout)
        if not os.path.exists(os.path.dirname('{}/2009_2018/'.format(area))):
            os.makedirs(os.path.dirname('{}/2009_2018/'.format(area)))
        plotly.offline.plot(fig, filename = '{}/2009_2018/corn correlation show for {}_2009.html'.format(area, fc),\
                        auto_open = False)

In [26]:
correlation_1999 = pd.DataFrame([])
correlation_2009 = pd.DataFrame([])
for state in states:
    correlation_1999[state] = corr_1999[state]['delta_yield_corn_{}'.format(state)].iloc[:-1]
    correlation_2009[state] = corr_2009[state]['delta_yield_corn_{}'.format(state)].iloc[:-1]
correlation_1999.to_csv('corn_correlation_1999.csv', index = False)
correlation_2009.to_csv('corn_correlation_2009.csv', index = False)

In [32]:
nw = ['north dakota']
w = ['south dakota', 'iowa', 'nebraska', 'kansas']
e = ['ohio', 'missouri', 'indiana', 'illinois']
sw = ['oklahoma', 'texas']
ne = ['michigan', 'minnesota', 'wisconsin']
se = ['north carolina']

In [33]:
plot_1999(w, 'w')
plot_2009(w, 'w')
plot_1999(nw, 'nw')
plot_2009(nw, 'nw')
plot_1999(e, 'e')
plot_2009(e, 'e')
plot_1999(sw, 'sw')
plot_2009(sw, 'sw')
plot_1999(ne, 'ne')
plot_2009(ne, 'ne')